In [ ]:
import os
import secrets
from PIL import Image
from flask import Flask, render_template, redirect, url_for, request, flash, session
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, LoginManager, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from flask_bcrypt import check_password_hash
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError
from wtforms.fields.html5 import DateField
from datetime import datetime
import re

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///user.db'
app.config['SECRET_KEY'] = "thisisasecretkey"
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(20), unique=True, nullable = False)
    password = db.Column(db.String(80), nullable = False)
    email = db.Column(db.String(50), nullable = False)
    bio = db.Column(db.Text)
    admin = db.Column(db.Boolean)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')
    notes = db.relationship('Note', backref='writer', lazy='dynamic')

class LoginForm(FlaskForm):
    username = StringField(label=u'用戶名稱', validators=[InputRequired(), Length(max=20)], render_kw={"placeholder":"用戶名稱"})
    password = PasswordField(label=u'密碼', validators=[InputRequired(), Length(max=20)], render_kw={"placeholder": "密碼"})
    submit = SubmitField("登入")
    
class Product(db.Model):
    __tablename__ = 'product'
    shop = db.Column(db.String(25))
    name = db.Column(db.Text, primary_key=True)
    oriprice = db.Column(db.Text)
    price = db.Column(db.Text)
    offer = db.Column(db.Text)
    shop_url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    desc = db.Column(db.Text)

class Subemail(db.Model):
    __tablename__ = 'subemail'
    subemail = db.Column(db.String(50), primary_key=True, nullable = False)

class Businesscop(db.Model):
    __tablename__ = 'businesscop'
    company = db.Column(db.String(100), primary_key=True, nullable = False)
    categories = db.Column(db.String(100), nullable = False)
    contact = db.Column(db.Integer,nullable = False)
    bemail = db.Column(db.String(100), nullable = False)
    tel = db.Column(db.Integer, nullable = False)
    
class Note(db.Model):
    __tablename__ = 'note'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    colour = db.Column(db.String(50))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.utcnow, nullable=False)
    
class NewNoteForm(FlaskForm):
    title = StringField("標題", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "標題"})
    colour = StringField("顏色", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "顏色"})
    note_body = StringField(u'記錄', widget=TextArea())
    date_created = DateField('DatePicker', format='%Y-%m-%d')
    submit = SubmitField("提交")

class UpdateNoteForm(FlaskForm):
    title = StringField("標題", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "標題"})
    # note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Note Body"})
    date_created = DateField('DatePicker', format='%Y-%m-%d')
    colour = StringField("顏色", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "顏色"})
    note_body = StringField(u'記錄', widget=TextArea())
    submit = SubmitField("更新")
    
class ProfileForm(FlaskForm):
    username = StringField('用戶名稱', validators=[InputRequired(), Length(min=2, max=20)])
    email = StringField("電郵地址", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    profile_body = StringField(u'Text', widget=TextArea())
    picture = FileField('更改您的個人頭像', validators=[FileAllowed(['jpg', 'png'])])
    submit = SubmitField('更新')

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))
    
@app.route("/", methods=["GET", "POST"])
def index():
    status = "home"
    if request.method == "POST":
        subemail = request.form['subemail']
        print(subemail)
        sub_email = Subemail(subemail=subemail)     
        try:
            db.session.add(sub_email)
            db.session.commit()
            print("Subscribed!")
            return redirect(url_for('index'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to subscribed.'
    else:
        return render_template('index.html',status=status)

@app.route("/member", methods=["GET", "POST"])
def member():
    status = "memberhome"
    if request.method == "POST":
        subemail = request.form['subemail']
        print(subemail)
        sub_email = Subemail(subemail=subemail)     
        try:
            db.session.add(sub_email)
            db.session.commit()
            print("Subscribed!")
            return redirect(url_for('member'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to subscribed.'
    else:
        return render_template('member.html',status=status)

@app.route("/register", methods=["GET", "POST"])
def register():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        password = bcrypt.generate_password_hash(password)
        print(username,email,password)
        new_user = User(username=username,email=email,password=password, admin=0)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('login'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html',status=status)

@app.route("/login", methods=["GET", "POST"])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for('member'))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route("/logout", methods=["GET", "POST"])
@login_required
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))

@app.route("/product")
@login_required
def product():
    sqlalchemyObj = db.engine.execute('select * from product ORDER BY Name')
    product = []
    for i in sqlalchemyObj:
        product.append(i)
    dataset = []
    dict={}
    for i in product:
        dict['Shop'] = i[0]
        dict['Name'] = i[1]
        dict['Oriprice'] = i[2]
        dict['Price'] = i[3]
        dict['Shop_url'] = i[4]
        dict['Img_url'] = i[5]
        dict['Desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) 
    print(dataset)
    
    return render_template('product.html', title='Product', dataset=dataset)

@app.route('/newrecord', methods=['GET','POST'])
@login_required
def newrecord():
    form = NewNoteForm()
    if form.validate_on_submit():
        print("record submitted")
        new_note = Note(title=form.title.data, colour=form.colour.data, date_created=form.date_created.data, note_body=form.note_body.data, writer=current_user, author=current_user.username)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('period'))
    return render_template('newrecord.html', title='New Note', form=form)

@app.route('/period', methods=['GET','POST'])
@login_required
def period():
    print('current user id:', current_user.id)
    if current_user.admin:
        # notes = Note.query.all()
        sqlalchemyObj = db.engine.execute('select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id ORDER BY user.id')
    else:
        # dataset = Note.query.filter_by(writer=current_user).all()
        sql = 'select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id WHERE note.note_writer == '+ str(current_user.id) + ' ORDER BY user.id'
        sqlalchemyObj = db.engine.execute(sql)
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
        # print(venues)
        dataset=[]
        dict={}
        for i in notes:
            dict['id'] = i[0]
            dict['title'] = i[1]
            dict['note_body'] = i[2]
            dict['date_created'] = i[3]
            dict['username'] = i[4]
            dict['image_file'] = i[5]
            # print(i)
            # print(dict)
            dataset.append(dict.copy())
    print(dataset)
    return render_template('period.html', notes=dataset, title='My Notes')

@app.route('/delete_record/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_record(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('period'))

@app.route('/update_record/<int:note_id>', methods=['GET',"POST"])
@login_required
def update_record(note_id):
    form = UpdateNoteForm()
    note = Note.query.get_or_404(note_id)
    if request.method == 'GET':
        form.title.data = note.title
        form.date_created.data = note.date_created
        form.colour.data = note.colour
        form.note_body.data = note.note_body
    if form.validate_on_submit():  # request.method="POST"
            note.query.filter_by(id=note_id).first()
            if note:
                note.title = form.title.data
                note.date_created = form.date_created.data
                note.colour = form.colour.data
                note.note_body = form.note_body.data
                db.session.commit()
                print("Note id=",note_id)
                print("Updated.")
                return redirect(url_for("period"))

    return render_template('update_record.html', form=form, title='Update Note', id=note_id)
    
@app.route("/profile", methods=['GET', 'POST'])
@login_required
def profile():
    form = ProfileForm()
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = save_picture(form.picture.data)
            current_user.image_file = picture_file
        current_user.username = form.username.data
        current_user.email = form.email.data
        current_user.bio = form.profile_body.data
        db.session.commit()
        flash('Your profile has been updated!', 'success')
        return redirect(url_for('profile'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.email.data = current_user.email
        form.profile_body.data = remove_html_tags(current_user.bio)
        if current_user.image_file is None:
            current_user.image_file = 'default.jpg'
    image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
    return render_template('profile.html', title='Profile', image_file=image_file, form=form)

def save_picture(form_picture):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)

    output_size = (125, 125)
    i = Image.open(form_picture)
    i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn

@app.route("/businesscop", methods=["GET", "POST"])
def businesscop():
    status = "businesscop"
    if request.method == "POST":
        company = request.form['company']
        categories = request.form['categories']
        contact = request.form['contact']
        bemail = request.form['bemail']
        tel = request.form['tel']
        business_partner = Businesscop(company=company,categories=categories,contact=contact, bemail=bemail, tel=tel)       
        try:
            db.session.add(business_partner)
            db.session.commit()
            print("Done!")
            return redirect(url_for('index'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to subscribed.'
    else:
        return render_template('businesscop.html',status=status)

@app.route("/acupoint")
def acupoint():
    return render_template('acupoint.html')

@app.route("/diet")
def diet():
    return render_template('diet.html')

@app.route("/drink")
def drink():
    return render_template('drink.html')

if __name__ == '__main__':
    app.run('localhost', 9004)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\User\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9004/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 19:47:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:47:16] "GET /static/img/hero-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 19:47:16] "GET /static/img/footer-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 19:47:17] "GET /acupoint HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:52:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:52:50] "GET /static/img/hero-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 19:52:51] "GET /static/img/footer-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2021 19:52:53] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:52:54] "GET /register HTTP/1.1" 